In [ ]:
#!pip install pypdf==3.17.1

In [ ]:

#!pip install numpy==1.26.4


In [ ]:
#!pip install bert_score

In [ ]:
#   pip install fpdf

In [ ]:
# Provide either INPUT_FILE path or INPUT_TEXT to summarize.
INPUT_FILE="examplee.pdf" # Insert file path here


# Style of summarization:

# Numbered List style
STYLE="Return your response as detailed summary which covers the main points of the text and key facts and figures."
PROMPT_TRIGGER="DETAILED SUMMARY"

# One sentence style
# STYLE="Return your response as one sentence which covers the main points of the text.",
# PROMPT_TRIGGER="ONE SENTENCE SUMMARY",

# Concise style
# STYLE="Return your response as concise summary which covers the main points of the text.",
# PROMPT_TRIGGER="CONCISE SUMMARY",

# Detailed style
# STYLE="Return your response as detailed summary which covers the main points of the text and key facts and figures.",
# PROMPT_TRIGGER="DETAILED SUMMARY",

# Output language, try e.g. Polish, Spanish, etc
OUTPUT_LANGUAGE = "Indonesia"

# Should output verbose info from underlying models, etc.
VERBOSE=True

In [ ]:
MODEL_CONTEXT_WINDOW=8192

# Maximal lenght of model's output, in tokens.
MAX_ANSWER_TOKENS = 4048

# Chunk params in characters (not tokens).
CHUNK_SIZE=10000
CHUNK_OVERLAP=500

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral-openorca")

response = llm.invoke("jonny have 3 bananas and he give lisa 1 banana. after that simon give jonny 2 bananas and 1 apple. what is jonny have?")

print(response)

C:\Users\User\AppData\Local\Temp\ipykernel_6564\666015976.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral-openorca")


 After Jonny gives Lisa 1 banana, he has 2 bananas left. Then Simon gives Jonny 2 bananas and 1 apple. So Jonny now has (3 - 1) + 2 + 1 = 4 bananas and 1 apple.


In [ ]:
from langchain.document_loaders import PyPDFLoader

def load_content():
    """Loads INPUT_FILE if set, otherwise returns INPUT_TEXT"""

    if INPUT_FILE:
        if INPUT_FILE.endswith(".pdf"):
            loader = PyPDFLoader(INPUT_FILE)
            docs = loader.load()
            print(f"PDF: loaded {len(docs)} pages")
            return "\n".join([d.page_content for d in docs])

        docs =  TextLoader(INPUT_FILE).load()
        return docs[0].page_content

    return INPUT_TEXT

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

combine_prompt_template = """
Write a summary of the following text delimited by tripple backquotes.
{style}

```{content}```

{trigger} in {language}:
"""

map_prompt_template = """
Write a concise summary of the following:
{text}

CONCISE SUMMARY in {language}:
"""

def summarize_base(llm, content):
    """Summarize whole content at once. The content needs to fit into model's context window."""

    prompt = PromptTemplate.from_template(
        combine_prompt_template
    ).partial(
        style=STYLE,
        trigger=PROMPT_TRIGGER,
        language=OUTPUT_LANGUAGE,
    )

    chain = LLMChain(llm=llm, prompt=prompt, verbose=VERBOSE)
    output = chain.run(content)

    return output


def summarize_map_reduce(llm, content):
    """Summarize content potentially larger that model's context window using map-reduce approach."""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )

    split_docs = text_splitter.create_documents([content])
    print(f"Map-Reduce content splits ({len(split_docs)} splits): {[len(sd.page_content) for sd in split_docs]}")

    map_prompt = PromptTemplate.from_template(
        map_prompt_template
    ).partial(
        language=OUTPUT_LANGUAGE,
    )

    combine_prompt = PromptTemplate.from_template(
        combine_prompt_template
    ).partial(
        style=STYLE,
        trigger=PROMPT_TRIGGER,
        language=OUTPUT_LANGUAGE,
    )

    chain = load_summarize_chain(
        llm=llm,
        chain_type="map_reduce",
        map_prompt=map_prompt,
        combine_prompt=combine_prompt,
        combine_document_variable_name="content",
        verbose=VERBOSE,
    )

    output = chain.run(split_docs)
    return output

In [ ]:
%%time

content = load_content()
content_tokens = llm.get_num_tokens(content)
print(f"Content length: {len(content)} chars, {content_tokens} tokens.")
print("Content sample:\n" + content[:200] + "\n\n")

# Keep part of context window for models output.
base_threshold = 0.75*MODEL_CONTEXT_WINDOW

if (content_tokens < base_threshold):
    print("Using summarizer: base")
    summary = summarize_base(llm, content)
else:
    print("Using summarizer: map-reduce")
    summary = summarize_map_reduce(llm, content)

print(f"Content length: {len(summary)} chars, {llm.get_num_tokens(summary)} tokens.")
print("Summary:\n" + summary + "\n\n")

PDF: loaded 11 pages
Content length: 46120 chars, 12289 tokens.
Content sample:
Natural Language Fine-Tuning
Jia Liu1,2,Yue Wang1,3,Zhiqi Lin1,3,Min Chen1,3∗,Yixue Hao2∗,Long Hu2∗
1Pazhou Laboratory, Guangzhou, China
2Huazhong University of Science and Technology, Wuhan, China
3S


Using summarizer: map-reduce
Map-Reduce content splits (5 splits): [9966, 9949, 9982, 9956, 8145]


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following:
Natural Language Fine-Tuning
Jia Liu1,2,Yue Wang1,3,Zhiqi Lin1,3,Min Chen1,3∗,Yixue Hao2∗,Long Hu2∗
1Pazhou Laboratory, Guangzhou, China
2Huazhong University of Science and Technology, Wuhan, China
3South China University of Technology, Guangzhou, China
{liujia0330, yixuehao, hulong }@hust.edu.cn,
{csyuewang, 202311089192 }@mail.scut.edu.cn,
minchen@ieee.org
Abstract
Large language model fine-tuning techniques typ-
ically depend on extensive labeled data, extern


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Write a summary of the following text delimited by tripple backquotes.
Return your response as detailed summary which covers the main points of the text and key facts and figures.

``` Kami menyusun algoritma pemantauan bahasa alam yang menganalisis probabilitas conditional token setiap token di berbagai kondisi prompt. Algoritma tersebut menggabungkan analisis langkah demi langkah dari pemantauan lembar jawaban, sehingga mempercepat proses pemodelan pemantauan model yang lebih efisien.

Penelitian ini memberikan dua kontribusi:
1. Paparnya menyajikan Natural Language Fine-Tuning (NLFT), sebuah algoritma pemodelan bertokentan. NLFT merupakan alternatif terhadap metode pemodelan yang menggunakan skor skala yang diperkenalkan oleh metode respon level sebelumnya.
2. Paparnya memproyeksikan algoritma minimal data yang tidak memerlukan tahap warm-up, yang diperlukan oleh proses ReFT. Hal ini meningkatkan akurasi

In [ ]:
 from fpdf import FPDF


def generate_pdf(filename, summary):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, summary)
    pdf.output(filename)
    print(f"File PDF '{filename}' berhasil dibuat.")

# Contoh penggunaan
summary_text = summary
generate_pdf("output_detail12.pdf", summary_text)

File PDF 'output_detail12.pdf' berhasil dibuat.


### Evaluation

##